# General
I'm very sorry that I changed the name of this kernel. This is becuase I often update this kernel and the content is changing.

Thanks to VESTA, we can know about what the addr2 data is.

https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#latest-596566

In this discussion(the URL↑), VESTA person says "addr2 as billing country".

Therefore, I want to identify the country and use data more efficiently.

I wish that this sharing help other kagglers!

# what already is known

I consulted with this discussion.

https://www.kaggle.com/c/ieee-fraud-detection/discussion/100400#579001

We can know the hour data from transactionDT.

Therefore, the gap of standard time should be supeculated.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.get_option("display.max_columns",500)

In [ ]:
folder_path = '../input/'
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')


df = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

## the appearance of addr2

In [ ]:
pd.DataFrame(df.groupby("addr2").count()["TransactionID"])

Various country data is used.

I use only the countries which have more tha 50 transaction data, because I can not guess the precise standard time if the amount of data is small.

That is, 16.0 31.0 32.0 60.0 65.0 87.0 96.0

### make hour column from transactionDT

As I said, I consulted with the discussion. Thanks for people who excite the discussion. 


In [ ]:
import datetime
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
df['TransactionDT'] = df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
df['hour'] = df['TransactionDT'].dt.hour

## speculation of standard time
At first, the country "87.0" has 520481 transaction.

This must be the U.S.

I checked the gap between the U.S. and others.

In [ ]:
df.groupby("addr2").mean()["hour"]

In [ ]:
df.groupby("addr2").mean()["hour"] - 14.049229

# 16,65
One of them may be Russia, but it is uncertain.

The possibility of china also exist.

However, we can say that these two countries are similar place to Russia or China.

I name them Asia group.

# 31,32
The standard time is similar to America.
Juding from country size, 32 should be Canada. 

However, it may be Mexico.

# 60,96
They are about -6 hour. 
Threfore, it should be Europe country.

# making future
At first, I made Europe future.

Soon, I'll increase it.

In [ ]:
def change(addr):
    if  addr==60.0:
        return 1
    elif addr==96.0:
        return 1
    elif addr == np.nan:
        return np.nan
    
    else:
        return 0

df["Europe"] = df["addr2"].map(change)

In [ ]:
df.groupby("Europe").count().isFraud

In [ ]:
586818*0.034585 + 3722*0.098872

In [ ]:
df.groupby("Europe").mean().isFraud

There is a gap between them.

In [ ]:
def change(addr):
    if  addr==16.0:
        return 1
    elif addr==65.0:
        return 1
    elif addr == np.nan:
        return np.nan
    
    else:
        return 0

df["Asia"] = df["addr2"].map(change)

In [ ]:
pd.DataFrame(df["Asia"]).info()

In [ ]:
df.groupby("Asia").isFraud.sum()

In [ ]:
df.groupby("Asia").isFraud.mean()

In [ ]:
def change(addr):
    if  addr==31.0:
        return 1
    elif addr==32.0:
        return 1
    elif addr == np.nan:
        return np.nan
    
    else:
        return 0

df["North America"] = df["addr2"].map(change)

In [ ]:
df.groupby("North America").isFraud.mean()

In [ ]:
plt.ylim(0,0.4)
plt.subplot(2,2,1)
plt.bar([0,1],df.groupby("Asia").mean().isFraud.values)


plt.subplot(2,2,2)
plt.ylim(0,0.4)
plt.bar([0,1],df.groupby("Europe").mean().isFraud.values)


plt.subplot(2,2,3)
plt.ylim(0,0.4)
plt.bar([0,1],df.groupby("North America").mean().isFraud.values)

We can find the difference between them.

# Conclusion
This identification is meningful to some degree, because we can find the gap between Europe data and other data.

The problem is that America has some standart time, so some error is appearing, for example, we can not boastfuly say that 16 is Russia.

I want to apply this result for futher data_engineering.